In [6]:
# import dependencies
from bs4 import BeautifulSoup as bs
import requests
import pymongo
from splinter import Browser
import pandas as pd
from selenium import webdriver

In [7]:
# initialize PyMongo to work with MongoDBs
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [8]:
# define database and collection
db = client.mars_db
collection = db.news

# NASA Mars News

Scrape the NASA Mars News Site and collect the latest News Title and Paragraph Text. Assign the text to variables that you can reference later.

In [29]:
# URL of MARS NEWS page to be scraped
url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

# SELENIUM
driver = webdriver.Chrome()
response = driver.get(url)
soup = bs(driver.page_source, "html.parser")
driver.quit()


In [30]:
# find title and paragraph
title = soup.find('div', class_='content_title')
print(title.text)

paragraph = soup.find('div', class_='article_teaser_body')
paragraph = paragraph.text
print(paragraph)

A Year of Surprising Science From NASA's InSight Mars Mission
A batch of new papers summarizes the lander's findings above and below the surface of the Red Planet.


# JPL Mars Space Images - Featured Image

-Visit the url for JPL Featured Space Image here.
-Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
-Make sure to find the image url to the full size .jpg image.
-Make sure to save a complete url string for this image.

In [31]:
#https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [32]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [33]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [34]:
html = browser.html
soup = bs(html, 'html.parser')

browser.click_link_by_partial_text('FULL IMAGE')
image = soup.find('a', class_="button fancybox")
image_url = image['data-fancybox-href']
firstparturl = 'https://www.jpl.nasa.gov/'
featured_image_url = firstparturl + image_url
print(featured_image_url)
browser.quit()

/Users/lorelley/opt/anaconda3/envs/PythonData/lib/python3.6/site-packages/splinter/driver/webdriver/__init__.py:504: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  FutureWarning,


https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA18840_ip.jpg


# Mars Weather

Visit the Mars Weather twitter account here and scrape the latest Mars weather tweet from the page. Save the tweet text for the weather report as a variable called mars_weather.

In [19]:
# URL of MARS WEATHER page to be scraped
url = "https://twitter.com/marswxreport?lang=en"

# retrieve page with requests module
response = requests.get(url)

# create Beautiful Soup object and parse
soup = bs(response.text, 'lxml')
#print(soup.prettify())

In [20]:
# scrape latest weather tweet
mars_weather = soup.find('p', class_='TweetTextSize')
mars_weather.text

'InSight sol 443 (2020-02-24) low -94.8ºC (-138.6ºF) high -12.3ºC (9.8ºF)\nwinds from the SSE at 7.0 m/s (15.6 mph) gusting to 22.2 m/s (49.6 mph)\npressure at 6.30 hPapic.twitter.com/aaRx6POMFO'

In [ ]:
# remove text from a tag


# Mars Facts

Visit the Mars Facts webpage here and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
Use Pandas to convert the data to a HTML table string.

In [21]:
# save url of MARS FACTS webpage
url = 'https://space-facts.com/mars/'

In [22]:
# extract tables from website
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [23]:
# create df
mars_df = tables[0]
mars_df.columns=['Category','MARS']


In [24]:
# convert df to html table string 
mars_html = mars_df.to_html()
mars_html.replace('\n','')
mars_html

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Category</th>\n      <th>MARS</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\n      <td>First Record:</td>\n      <td>2nd millennium BC</

In [25]:
# save html to file and hide index
mars_df.to_html('mars_facts.html', index=False)
!open mars_facts.html

# Mars Hemispheres

*Visit the USGS Astrogeology site here to obtain high resolution images for each of Mar's hemispheres.
*You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.
*Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
*Append the dictionary with the image url string and the hemisphere title to a list. This list will contain one dictionary for each hemisphere.

In [ ]:
# SELENIUM
url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
driver = webdriver.Chrome()
response = driver.get(url)
soup = bs(driver.page_source, "html.parser")


In [26]:
hemisphere_image_urls = []
y = 0

# print title of each hemisphere in for loop
for each in list(soup.find_all('div', class_='item')):
    title = each.find('h3').text
    
    for x in range(4):
        x += y
        driver.find_element_by_xpath(f'//*[@id="product-section"]/div[2]/div[{x+1}]/a/img').click()    
        driver.find_element_by_xpath('//*[@id="wide-image-toggle"]').click()

        # print urls
        img = driver.find_element_by_xpath('//*[@id="wide-image"]/div/ul/li[1]/a')
        img_url = img.get_attribute('href')

        hemispheres = {"title": title, "img_url": img_url}
        hemisphere_image_urls.append(hemispheres)
        break
        
    driver.back()
    y += 1    
       
print(hemisphere_image_urls)
driver.quit()

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
